### **گام سوم**

## آماده سازی کولب

In [1]:
# connecting to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# !rm -r spark*
!ls

gdrive	sample_data


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
sc = spark.sparkContext
spark

## آماده‌سازی داده‌ها



ابتدا داده‌های بورسی مربوط بازه‌ی دهم فروردین ۱۴۰۰ الی ۱۱ خرداد ۱۴۰۰ را توسط یک کد پایتون دانلود می‌کنیم و آن‌هایی که خالی هستند را با همان کد حذف می‌کنیم. سپس یک پوشه شامل داده‌های بورسی را در گوگل درایو آپلود می‌کنیم. حال به کمک اسپارک اولین فایل حاوی داده‌های روزانه بورسی را باز می‌کنیم پس از انجام پردازش‌های لازم، آن را درونن یک دیتافریم میریزیم. سپس فایل‌های بورسی را دانه دانه باز می‌کنیم و پس از انجام پردازش‌های لازم،‌ آن‌ را به دیتافریم از پیش یاخته شده اظافه می‌کنیم. در نهایت همه‌ی داده‌های بورسی درون یک دیتافریم ذخیره می‌شوند

In [6]:
''' at first, loading the first csv file in folder of "stock_daily_data_csv" into single dataframe.
Then, add columns of day, month, year to the dataframe named "df". after that, 
loading the other dataframes by name of "df2" and then join that to df.
'''
# a function to get the file path as input and return the day, month and year as output.
def get_dat(file_paths):    
  date = file_paths.split('/')[-1].split('.')[0] 
  # getting the day, month and year by using split
  year, month, day = date.split('-')[0], date.split('-')[1], date.split('-')[-1]
  return  int(year), int(month), int(day)

# using glob to put paths of all csv files in folder of "stock_daily_data_csv".
import glob
file_paths = glob.glob('/content/gdrive/My Drive/bigData_hw3/Q3/stock_daily_data_csv' + '/*.csv')

# creating dataFrame as df by using path of first csv file; then, adding the other csv files to the current df.
df = spark.read.csv(file_paths[0], inferSchema =True, header = True)
# adding columns of day, month, year to the dataframe
from pyspark.sql.functions import lit
df = df.withColumn("year", lit( get_dat(file_paths[0])[0] )).withColumn("month", lit( get_dat(file_paths[0])[1] )).withColumn("day", lit( get_dat(file_paths[0])[2] ))

# adding other dataframes to df.
for element in file_paths[1:]:
  df2 = spark.read.csv(element, inferSchema =True, header = True)
  # adding columns of day, month, year to the dataframe
  df2 = df2.withColumn("year", lit( get_dat(element)[0] )).withColumn("month", lit( get_dat(element)[1] )).withColumn("day", lit( get_dat(element)[2] ))
  # union main dataframe named "df" and newly created dataframe named "df2" 
  df = df.union(df2)

# showing loaded data
print('size of dataframe:   ', (df.count(), len(df.columns)))
df

size of dataframe:    (36752, 19)


_c0,نماد,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,year,month,day
2,پالايش,صندوق پالايشي يکم...,32817,31306160,2553721022480,81440,80500,81300,-140,-0.17,81570,130,0.16,79090,83430,1400,1,10
3,شپنا,پالايش نفت اصفهان,25980,162908660,2288555785720,14140,13860,13900,-240,-1.7,14050,-90,-0.64,13860,14570,1400,1,10
4,شستا,سرمايه گذاري تامي...,24807,83528311,961348742900,11440,11520,11440,0,0.0,11510,70,0.61,11220,11770,1400,1,10
5,سپيدار,سپيدار سيستم آسيا,22322,2070784,133043763409,65403,64510,65340,-63,-0.1,64248,-1155,-1.77,64095,65960,1400,1,10
6,شتران,پالايش نفت تهران,21418,179232061,1842881877510,10050,9900,10330,280,2.79,10280,230,2.29,9850,10590,1400,1,10
7,دارا يكم,صندوق واسطه گري م...,18461,10645997,1545689341020,145340,144100,143980,-1360,-0.94,145190,-150,-0.1,143220,147250,1400,1,10
8,اپال,فرآوري معدني اپال...,16782,24209820,438341590800,17380,17450,18240,860,4.95,18110,730,4.2,17450,18290,1400,1,10
9,زملارد,كشاورزي و دامپرور...,16269,4503175,177657700682,37711,38420,39973,2262,6.0,39452,1741,4.62,36960,39973,1400,1,10
10,فملي,ملي‌ صنايع‌ مس‌ ا...,14067,56622241,738306083250,13230,12990,13010,-220,-1.66,13040,-190,-1.44,12970,13400,1400,1,10
11,فولاد,فولاد مباركه اصفهان,11833,55662597,777505009350,13800,13640,13930,130,0.94,13970,170,1.23,13600,14300,1400,1,10


# spark dataframe

## بخش اول

### (۱۰ نماد)ارزان‌ترین نماد بورسی

با توجه به اینکه از ما خواسته شده است ۱۰ تا از ارزان‌ترین نماد‌های بورسی آخرین روز را بیابیم، ابتدا به کمک فیلتر روی مقدار روز و ماه، داده‌های روز آخر را در نظر می‌گیریم. سپس آن‌ها را با توجه به مقادیر ستون "قیمت پایانی - مقدار" سورت می‌کنیم. نتیجه در خروجی سلول بعدی قابل مشاهده است

In [7]:
# getting data of the last day by using filter.
df_last_day = df.filter( (df.day  == 11 ) & (df.month  == 3) )

# sorting by column of "قیمت پایانی - مقدار" and get ten cheapest stock symbol.
df_ten_cheapest = df_last_day.orderBy("قیمت پایانی - مقدار" , ascending=True).take(10)

# convverting to spark dataFrame in order to better showing
df_ten_cheapest = spark.createDataFrame(df_ten_cheapest)
print(" ten cheapest stock symbol in the 11th of khordad:", '\n')
df_ten_cheapest

 ten cheapest stock symbol in the 11th of khordad: 



_c0,نماد,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,year,month,day
448,هپارس102,اختيارف ت پارس-17...,27,104674,104674,1,1,1,0,0.0,1,0,0.0,1,1,1400,3,11
679,ضسپا3005,اختيارخ خساپا-300...,2,200,200000,2,1,1,-1,-50.0,1,-1,-50.0,1,1,1400,3,11
468,هنوري102,اختيارف ت نوري737...,21,300000,300000,1,1,1,0,0.0,1,0,0.0,1,1,1400,3,11
171,هشستا103,اختيارف ت شستا-11...,572,2645644,2645644,1,1,1,0,0.0,1,0,0.0,1,1,1400,3,11
505,عيار2,صندوق س. پشتوانه ...,12,120000,120000,26915,1,1,-26914,-100.0,1,-26914,-100.0,1,1,1400,3,11
596,آكورد2,صندوق س. آرمان آت...,5,500000,500000,32347,1,1,-32346,-100.0,1,-32346,-100.0,1,1,1400,3,11
325,دارا2,صندوق س. دارا الگ...,145,14500000,14500000,10020,1,1,-10019,-99.99,1,-10019,-99.99,1,1,1400,3,11
655,افران2,صندوق س افرا نماد...,3,8000000,8000000,13422,1,1,-13421,-99.99,1,-13421,-99.99,1,1,1400,3,11
654,كمند2,صندوق س. با درآمد...,3,105000000,105000000,10143,1,1,-10142,-99.99,1,-10142,-99.99,1,1,1400,3,11
361,فردا2,صندوق س آواي فردا...,100,10000000,10000000,10781,1,1,-10780,-99.99,1,-10780,-99.99,1,1,1400,3,11


### گران‌ترین نماد بورسی (۱۰نماد)

برای این قسمت همانند یافتن ۱۰ تا از ارزان‌ترین نماد‌های بورسی عمل می‌کنیم با این تفاوت که  ترتیب سورت کردنمون نزولی خواهد بود. نتیجه در خروجی سلول بعد قابل مشاهده است

In [8]:
# sorting by column of "قیمت پایانی - مقدار" and get ten most expensive stock symbols.
df_ten_most_expensive = df_last_day.orderBy("قیمت پایانی - مقدار" , ascending=False ).take(10)

# convverting to spark dataFrame in order to better showing
df_ten_most_expensive = spark.createDataFrame(df_ten_most_expensive)
print(" ten most expensive stock symbols in the 11th of khordad:", '\n')
df_ten_most_expensive


 ten most expensive stock symbols in the 11th of khordad: 



_c0,نماد,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,year,month,day
560,پست0008پ08,پسته فندقي32-30مه...,7,408,679339394,1699960,1680000,1647097,-52863,-3.11,1665047,-34913,-2.05,1647097,1740000,1400,3,11
763,پست0008پ04,پسته فندقي32-30سب...,1,4,6588388,1734242,1647097,1647097,-87145,-5.02,1647097,-87145,-5.02,1647097,1647097,1400,3,11
559,سنفت007,سلف موازي استاندا...,7,675000,1064561400000,1593523,1577128,1577128,-16395,-1.03,1577128,-16395,-1.03,1577128,1577128,1400,3,11
717,سميعا002,سلف موازي استاندا...,2,153680,201715603920,1312005,1312569,1312569,564,0.04,1312569,564,0.04,1312569,1312569,1400,3,11
765,سنفت001,سلف موازي استاندا...,1,17200,20049438000,1165665,1165665,1165665,0,0.0,1165665,0,0.0,1165665,1165665,1400,3,11
436,سكه0012پ04,سكه تمام بهارتحوي...,37,6400,6916087200,1061562,1078000,1082990,21428,2.02,1080639,19077,1.8,1077000,1084989,1400,3,11
434,سكه0112پ03,سكه تمام بهارتحوي...,38,7700,8320460100,1063820,1078000,1083000,19180,1.8,1080579,16759,1.58,1077300,1083000,1400,3,11
257,سكه0012پ01,سكه تمام بهارتحوي...,263,107100,115678480800,1061940,1078000,1082000,20060,1.89,1080098,18158,1.71,1072200,1083500,1400,3,11
585,سكه0111پ05,سكه تمام بهارتحوي...,6,1000,1078751200,1048992,1075000,1080000,31008,2.96,1078751,29759,2.84,1075000,1086500,1400,3,11
379,سكه0011پ02,سكه تمام بهارتحوي...,84,30200,32569439800,1062421,1081300,1083200,20779,1.96,1078458,16037,1.51,1072500,1083500,1400,3,11


## بخش دوم

در اینجا هم از سورت کردن روی ستون حجم استفاده می‌کنیم. سپس ردیفی که بزرگترین حجم را دارد به عنوان مطلوب سوال نمایش می‌هیم

In [34]:
# sorting by column of "حجم" and getting stock symbol with biggest volume.
df_voluminous_symbol = df.orderBy("حجم" , ascending=False).take(1)

# converting to spark dataFrame in order to better showing
df_voluminous_symbol = spark.createDataFrame(df_voluminous_symbol)
print("Most voluminous stock symbol:", '\n')
df_voluminous_symbol

Most voluminous stock symbol: 



_c0,نماد,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,year,month,day
494,خودرو4,ايران‌ خودرو,47,22973995436,54678109137680,2650,2380,2380,-270,-10.19,2380,-270,-10.19,2380,2380,1400,1,10


## بخش سوم

در این بخش، من بیشترین تغیر قیمت را پیدا کردم و نتوانستم بیشترین افزایش و کاهش قیمت را بیابم. از این جهت سلول پاسخ بخش ۳ و ۴ خواهد بود. برای حل این سوال از سه ویندو فانکشن استفاده می‌کنیم. توضیحات تکمیلی در کامنت‌های کد موجود است. در ویندو فانکشن اول برای هر ماه و هر نماد، یک ستون به عنوان بیشترین تغییر قیمت اظافه می‌کنیم. به کمک ویندو فانکش دوم، برای هر ماه و هر نماد، فقط آن ردیفی را درنظر می‌گیریم که بیشترین تغیر قیمت را دارد. یعنی الان، برای هر ماه، بیشترین تغیر قیمت هر نماد را داریم. سپس با وینپو فانکشن سوم، در هر ماه، ۱۰ تا از نمادهایی که بیشترین تغیر قیمت را داشته ان را برمی‌گردانیم. خروجی در نتیجه سلول بعدی نمایش داده شده است

In [ ]:
'''   we use three window in order to solve this part. by using first window named "window_add_difference",
we partition data by "month", 'نماد' . then, order in by "قیمت پایانی - مقدار". finally, we add a new column named "تغیر قیمت".
by using second window named "window_to_get_max_diff", 
we get just the maximum row in each grouped data (this row is maximum difference for each month-symbol).
third window named "window_to_get_10_max_diff" is used to get 10 maximum row in each grouped data.

 NOTE: 
* order in 1st and 2nd window is different.
** the partition in 2nd window and 3rd is different.    '''

import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import *

# window one: partitioning data by "month", 'نماد' . then, order by "قیمت پایانی - مقدار". 
window_add_difference = Window.partitionBy("month", 'نماد').orderBy(col("قیمت پایانی - مقدار").desc())

# adding a new column named "تغیر قیمت"
df_dif_month_symbol = df.select('month', 'نماد', 
                    (F.max("قیمت پایانی - مقدار").over(window_add_difference) - F.min("قیمت پایانی - مقدار").over(window_add_difference))
                    .alias('تغیر قیمت'))

# window two: partitioning data by "month", 'نماد' . then, order by "تغیر قیمت". 
window_to_get_max_diff = Window.partitionBy("month",'نماد').orderBy(col('تغیر قیمت').desc())

# by adding a new column named "row" by exerting row_number(),
# we try to get maximum of each grouped data. then, we eliminate the column named "row".
df_max_dif_mounth_symbol = df_dif_month_symbol.withColumn("row", row_number().over(window_to_get_max_diff)
                                                                            ).filter(col("row") == 1 ).drop('row')

# window three: in order to get 10 maximum row in each grouped data. again, we exert row_number().
window_to_get_10_max_diff = Window.partitionBy("month").orderBy(col('تغیر قیمت').desc())

# same as window two, we get 10 maximum row in each grouped data.
df_max_dif_mounth = df_max_dif_mounth_symbol.withColumn("row",row_number().over(window_to_get_10_max_diff)
                                                                         ).filter(col("row") <= 10 ).drop('row')

# saving
df_max_dif_mounth.toPandas().to_csv('/content/gdrive/My Drive/bigData_hw3/Q3/result_Q3_3.csv', header=True)
print('result have been saved in a file named "result_Q3_3.csv" in colab', '---'*20, '\n')
print('some part of the result is printed in below:')
df_max_dif_mounth

result have been saved in a file named "result_Q3_3.csv" in colab ------------------------------------------------------------ 

some part of the result is printed in below:


month,نماد,تغیر قیمت
1,پست0008پ09,272249
1,تملي804,261280
1,تملي806,260339
1,تملي705,258693
1,پست0008پ04,248727
1,آرماني,231000
1,تملي710,224357
1,پست0008پ08,205527
1,پست0008پ02,175612
1,تملي701,154823


## بخش پنجم

نمادهایی را بسته در نظر گرفته‌ام که حجم کمتر از ۲۰ داشتند

In [52]:
# any symbol with volum less than 20 is considered as closed in that day.
df_colsed = df.filter(col('حجم') <= 20 ).groupBy('نماد').count().withColumnRenamed("count","تعداد بسته بودن").orderBy('تعداد بسته بودن', ascending=False).take(1)
spark.createDataFrame(df_colsed)

نماد,تعداد بسته بودن
ضمخا7066,18


# spark SQL

## تبدیل نام فارسی ستون‌‌ها به نام انگلیسی

از آن جهت که می‌خواهیم کوئری بزنیم، نام فارسی ستون‌ها را تغییر می‌دهیم. سپس به کمک کد زیر نمای جدولی از دیتافریم را می‌سازیم تا بتوانیم از کوئری استفاده کینم



In [26]:
# renaming column names of df
newColumns = ['_c0', "symbol", "name", "number", "volume", "value", "yesterday", "first", "last_trade_amount",
              "last_trade_diff", "last_trade_precent", "final_price_amount", "final_price_diff",
              "final_price_percent", "least", "most", "year", "month", "day"]
df_renamed_column = df.toDF(*newColumns)

# creating a table view of the dataframe in order to query on it
df_renamed_column.createOrReplaceTempView('table_view_of_df')

# showing the renamd dataframe
spark.createDataFrame(df_renamed_column.take(5))

_c0,symbol,name,number,volume,value,yesterday,first,last_trade_amount,last_trade_diff,last_trade_precent,final_price_amount,final_price_diff,final_price_percent,least,most,year,month,day
2,پالايش,صندوق پالايشي يکم...,32817,31306160,2553721022480,81440,80500,81300,-140,-0.17,81570,130,0.16,79090,83430,1400,1,10
3,شپنا,پالايش نفت اصفهان,25980,162908660,2288555785720,14140,13860,13900,-240,-1.7,14050,-90,-0.64,13860,14570,1400,1,10
4,شستا,سرمايه گذاري تامي...,24807,83528311,961348742900,11440,11520,11440,0,0.0,11510,70,0.61,11220,11770,1400,1,10
5,سپيدار,سپيدار سيستم آسيا,22322,2070784,133043763409,65403,64510,65340,-63,-0.1,64248,-1155,-1.77,64095,65960,1400,1,10
6,شتران,پالايش نفت تهران,21418,179232061,1842881877510,10050,9900,10330,280,2.79,10280,230,2.29,9850,10590,1400,1,10


## بخش اول

### (۱۰ نماد)ارزان‌ترین نماد بورسی

In [31]:
sql_ten_cheapest = spark.sql('''  
  SELECT * FROM table_view_of_df  
  WHERE month = 3 and day = 11
  order by final_price_amount ASC limit 10
                             ''')
print(" ten cheapest stock symbol in the 11th of khordad:", '\n')
sql_ten_cheapest

 ten cheapest stock symbol in the 11th of khordad: 



_c0,symbol,name,number,volume,value,yesterday,first,last_trade_amount,last_trade_diff,last_trade_precent,final_price_amount,final_price_diff,final_price_percent,least,most,year,month,day
448,هپارس102,اختيارف ت پارس-17...,27,104674,104674,1,1,1,0,0.0,1,0,0.0,1,1,1400,3,11
679,ضسپا3005,اختيارخ خساپا-300...,2,200,200000,2,1,1,-1,-50.0,1,-1,-50.0,1,1,1400,3,11
468,هنوري102,اختيارف ت نوري737...,21,300000,300000,1,1,1,0,0.0,1,0,0.0,1,1,1400,3,11
171,هشستا103,اختيارف ت شستا-11...,572,2645644,2645644,1,1,1,0,0.0,1,0,0.0,1,1,1400,3,11
505,عيار2,صندوق س. پشتوانه ...,12,120000,120000,26915,1,1,-26914,-100.0,1,-26914,-100.0,1,1,1400,3,11
596,آكورد2,صندوق س. آرمان آت...,5,500000,500000,32347,1,1,-32346,-100.0,1,-32346,-100.0,1,1,1400,3,11
325,دارا2,صندوق س. دارا الگ...,145,14500000,14500000,10020,1,1,-10019,-99.99,1,-10019,-99.99,1,1,1400,3,11
655,افران2,صندوق س افرا نماد...,3,8000000,8000000,13422,1,1,-13421,-99.99,1,-13421,-99.99,1,1,1400,3,11
654,كمند2,صندوق س. با درآمد...,3,105000000,105000000,10143,1,1,-10142,-99.99,1,-10142,-99.99,1,1,1400,3,11
361,فردا2,صندوق س آواي فردا...,100,10000000,10000000,10781,1,1,-10780,-99.99,1,-10780,-99.99,1,1,1400,3,11


### گران‌ترین نماد بورسی (۱۰نماد)

In [32]:
sql_ten_most_expensive = spark.sql('''  
  SELECT * FROM table_view_of_df  
  WHERE month = 3 and day = 11
  order by final_price_amount DESC limit 10
                                  ''')
print(" ten most expensive stock symbol in the 11th of khordad:", '\n')
sql_ten_most_expensive

 ten most expensive stock symbol in the 11th of khordad: 



_c0,symbol,name,number,volume,value,yesterday,first,last_trade_amount,last_trade_diff,last_trade_precent,final_price_amount,final_price_diff,final_price_percent,least,most,year,month,day
560,پست0008پ08,پسته فندقي32-30مه...,7,408,679339394,1699960,1680000,1647097,-52863,-3.11,1665047,-34913,-2.05,1647097,1740000,1400,3,11
763,پست0008پ04,پسته فندقي32-30سب...,1,4,6588388,1734242,1647097,1647097,-87145,-5.02,1647097,-87145,-5.02,1647097,1647097,1400,3,11
559,سنفت007,سلف موازي استاندا...,7,675000,1064561400000,1593523,1577128,1577128,-16395,-1.03,1577128,-16395,-1.03,1577128,1577128,1400,3,11
717,سميعا002,سلف موازي استاندا...,2,153680,201715603920,1312005,1312569,1312569,564,0.04,1312569,564,0.04,1312569,1312569,1400,3,11
765,سنفت001,سلف موازي استاندا...,1,17200,20049438000,1165665,1165665,1165665,0,0.0,1165665,0,0.0,1165665,1165665,1400,3,11
436,سكه0012پ04,سكه تمام بهارتحوي...,37,6400,6916087200,1061562,1078000,1082990,21428,2.02,1080639,19077,1.8,1077000,1084989,1400,3,11
434,سكه0112پ03,سكه تمام بهارتحوي...,38,7700,8320460100,1063820,1078000,1083000,19180,1.8,1080579,16759,1.58,1077300,1083000,1400,3,11
257,سكه0012پ01,سكه تمام بهارتحوي...,263,107100,115678480800,1061940,1078000,1082000,20060,1.89,1080098,18158,1.71,1072200,1083500,1400,3,11
585,سكه0111پ05,سكه تمام بهارتحوي...,6,1000,1078751200,1048992,1075000,1080000,31008,2.96,1078751,29759,2.84,1075000,1086500,1400,3,11
379,سكه0011پ02,سكه تمام بهارتحوي...,84,30200,32569439800,1062421,1081300,1083200,20779,1.96,1078458,16037,1.51,1072500,1083500,1400,3,11


## بخش دوم

In [33]:
sql_voluminous_symbol = spark.sql("""
  SELECT * FROM table_view_of_df
  order by volume DESC limit 1
                                 """)
print("Most voluminous stock symbol:", '\n')
sql_voluminous_symbol

Most voluminous stock symbol: 



_c0,symbol,name,number,volume,value,yesterday,first,last_trade_amount,last_trade_diff,last_trade_precent,final_price_amount,final_price_diff,final_price_percent,least,most,year,month,day
494,خودرو4,ايران‌ خودرو,47,22973995436,54678109137680,2650,2380,2380,-270,-10.19,2380,-270,-10.19,2380,2380,1400,1,10


## بخش پنجم

In [50]:
sql_colsed = spark.sql("""
  select symbol, number_of_closed from 
  ( SELECT symbol, COUNT(*) as number_of_closed FROM table_view_of_df as table WHERE table.volume <= 20 GROUP BY symbol)
                            as table2 ORDER BY number_of_closed DESC LIMIT 1
                       """)
sql_colsed

symbol,number_of_closed
ضمخا7066,18
